# EQ2425 - Analysis and Search of Visual Data

## Project 3: Image Classification using CNNs

**Author: Yage Hao (yage@kth.se)**

### Import packages

In [1]:
import numpy as np

import torch
import torch.nn.functional as F
import torch.nn as nn 
import torch.optim as optim 
import torchvision
import torchvision.transforms as transforms

from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

import statistics 

from sklearn.metrics import recall_score

In [2]:
print('PyTorch version:', torch.__version__)
print('Torchvision version:', torchvision.__version__)

PyTorch version: 1.9.0+cu111
Torchvision version: 0.10.0+cu111


### Data Pre-processing

In [3]:
normalize = transforms.Normalize(mean=[.5, .5, .5],std=[1 ,1, 1])

dataset_transform = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

In [4]:
train_set = CIFAR10('./cifar10', train=True, download=True, transform=dataset_transform)
valid_set = CIFAR10('./cifar10', train=False, download=True, transform=dataset_transform)

print(train_set.data.shape)
print(valid_set.data.shape)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar10/cifar-10-python.tar.gz to ./cifar10
Files already downloaded and verified
(50000, 32, 32, 3)
(10000, 32, 32, 3)


In [5]:
train_loader = DataLoader(train_set, batch_size=64, num_workers=0, shuffle=False)
valid_loader = DataLoader(valid_set, batch_size=64, num_workers=0, shuffle=False)

### Build the Default Model

In [6]:
class BasicCNN(nn.Module):

  def __init__(self, num_classes):
    super(BasicCNN, self).__init__()
    self.conv1 = nn.Conv2d(3, 24, (5,5), stride=1, padding=0) 
    self.conv2 = nn.Conv2d(24, 48, (3,3), stride=1, padding=0) 
    self.conv3 = nn.Conv2d(48, 96, (3,3), stride=1, padding=0) 
    self.pool1 = nn.MaxPool2d((2,2), stride=2)
    self.pool2 = nn.MaxPool2d((2,2), stride=2)
    self.pool3 = nn.MaxPool2d((2,2), stride=2)
    self.fc1 = nn.Linear(2*2*96, 512)
    self.fc2 = nn.Linear(512, 10)
    self.sm = nn.Softmax() 
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.pool1(x)
    x = F.relu(self.conv2(x))
    x = self.pool2(x)
    x = F.relu(self.conv3(x))
    x = self.pool3(x)
    x = x.reshape(x.size(0), -1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    x = self.sm(x)
    return x 

In [7]:
cuda = torch.device('cuda')

model = BasicCNN(10)
model.to(cuda)

BasicCNN(
  (conv1): Conv2d(3, 24, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(24, 48, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(48, 96, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool2): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (pool3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=384, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
  (sm): Softmax(dim=None)
)

### Define functions for training

Optimizer: SGD

In [8]:
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

Train function

In [9]:
def train(model, loss_fn, optimizer):
  model.train()

  train_batch_losses = []

  for batch, labels in train_loader:
    batch = batch.to(cuda)
    labels = labels.to(cuda)

    optimizer.zero_grad()
    y_pred = model(batch)
    loss = loss_fn(y_pred, labels)
    loss.backward()
    optimizer.step()

    train_batch_losses.append(float(loss))

    mean_loss = statistics.mean(train_batch_losses)
  
  return mean_loss

Validation function

In [10]:
def validate(model, loss_fn, optimizer):
  model.eval()

  predictions = []

  with torch.no_grad():
    validation_batch_losses = []

    for batch, labels in valid_loader:
      batch = batch.to(cuda)
      labels = labels.to(cuda)

      labels_pred = model(batch)
      loss = loss_fn(labels_pred, labels)

      validation_batch_losses.append(float(loss))

      mean_loss = statistics.mean(validation_batch_losses)
  return mean_loss

Recall Rate


In [11]:
def recall(model, loader):
  model.eval()

  with torch.no_grad():
    batch_recall = []

    for batch, labels in loader:
      batch = batch.to(cuda)
      labels = labels.to(cuda)

      labels_pred = model(batch)
      _, predicted = torch.max(labels_pred.cpu().data, 1)

      ma_recall = recall_score(labels.cpu(), predicted.cpu(), average='macro')
      
      batch_recall.append(ma_recall)
      mean_recall = statistics.mean(batch_recall)

  return mean_recall

### Training

In [ ]:
loss_fn = nn.CrossEntropyLoss()

epochs = 300

train_losses = []
valid_losses = []

train_recalls = []
valid_recalls = []

for epoch in range(1, epochs+1):
  print('Epoch:', epoch)

  train_loss = train(model, loss_fn, optimizer)
  train_recall = recall(model, train_loader)
  train_losses.append(train_loss)
  train_recalls.append(train_recall)

  print('Training loss:', train_loss)
  print('Training recall: {}%'.format(train_recall))

  valid_loss = validate(model, loss_fn, optimizer)
  valid_recall = recall(model, valid_loader)
  valid_losses.append(valid_loss)
  valid_recalls.append(valid_recall)

  print('Validation loss:', valid_loss)
  print('Validation recall: {}%'.format(valid_recall))

Epoch: 1


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training loss: 2.302545941394308
Training recall: 0.11643774407840904%
Validation loss: 2.302526943243233
Validation recall: 0.11407118359347658%
Epoch: 2


In [ ]:
print('Average training recall: {}%'.format(statistics.mean(train_recalls)))
print('Average validation recall: {}%'.format(statistics.mean(valid_recalls)))

Result Document: https://docs.google.com/spreadsheets/d/1ojYF_o1rZ-KGAufCUG6QpHWw0i8j_2-ur9olJV41dvo/edit?usp=sharing

ref:


*   https://colab.research.google.com/github/ccarpenterg/LearningPyTorch1.x/blob/master/04_cifar_10_challenging_convnets.ipynb#scrollTo=YczNCGVTJXp9
*   https://blog.csdn.net/u013347145/article/details/104332094

